# 🎮 Trabajo Final - Consulta Inteligente sobre Jugadores del FIFA 23

Este trabajo final integra técnicas de procesamiento de lenguaje natural con modelos de embeddings y un LLM para permitir consultas en lenguaje natural sobre datos de jugadores del FIFA 23.

🧾 Los datos fueron obtenidos de un archivo CSV con estadísticas detalladas de cada jugador como:
- Nombre y apellido
- Liga y equipo que juega
- Pais de origen
- Estadísticas ofensivas y defensivas (ritmo, fisico, tiro, defensa, etc.)

⚙️ La app utiliza:
- **HuggingFace Embeddings** para vectorizar los datos (GPU si está disponible)
- **Gemini (Google)** como modelo de lenguaje para responder consultas
- **LlamaIndex** para realizar búsquedas semánticas sobre los datos
- **Streamlit** para la interfaz interactiva

## Instalacion de dependencias

In [ ]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-llms-langchain langchain-community langchain-google-genai google-generativeai pandas torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "" #PONER API KEY DE GOOGLE

## Lectura y limpieza del dataset

In [ ]:
from collections import defaultdict
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')
# ruta_csv = "/content/drive/MyDrive/Trabajo Final Inteligentes - Juan/jugadores_filtrados.csv"

# Cargar jugadores
df = pd.read_csv("jugadores_filtrados.csv")
df.head()

,id_jugador,nombre_corto,nombre_largo,posiciones_jugador,valoracion_general,potencial,valor_eur,salario_eur,edad,fecha_nacimiento,...,contencion_izq,contencion,contencion_der,lateral_der_ofensivo,lateral_izq,central_izq,central,central_der,lateral_der,arquero
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,RW,91,91,54000000.0,195000.0,35,1987-06-24,...,63+3,63+3,63+3,64+3,59+3,50+3,50+3,50+3,59+3,19+3
1,165153,K. Benzema,Karim Benzema,"CF, ST",91,91,64000000.0,450000.0,34,1987-12-19,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3
2,188545,R. Lewandowski,Robert Lewandowski,ST,91,91,84000000.0,420000.0,33,1988-08-21,...,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
3,192985,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,107500000.0,350000.0,31,1991-06-28,...,79+3,79+3,79+3,78+3,74+3,68+3,68+3,68+3,74+3,21+3
4,231747,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,190500000.0,230000.0,23,1998-12-20,...,63+3,63+3,63+3,67+3,63+3,54+3,54+3,54+3,63+3,18+3


##Transformacion a documentos de texto (no ejecutar si cargamos el index)

In [ ]:
from llama_index.core import VectorStoreIndex, Document

# Convertir cada fila a un documento de texto para indexar
documents = []
for _, row in df.iterrows():
    text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    documents.append(Document(text=text))

## Preparacion del modelo de embeddings

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch
# Usar embeddings con GPU si hay disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",#all-mpnet-base-v2/all-MiniLM-L12-v2
    device=device
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Configuracion del modelo de lenguaje (LLM)

In [ ]:
from llama_index.core import Settings
from langchain_google_genai import ChatGoogleGenerativeAI
# Cliente Gemini como LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Crear contexto de servicio
Settings.llm = llm
Settings.embed_model = embed_model

## Creando indice (no ejecutar si cargamos el index)

In [ ]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top_k=3)

## Guardar el indice (no ejecutar si cargamos el index)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
persist_dir = "/content/drive/MyDrive/indice_jugadores_base2"
index.storage_context.persist(persist_dir=persist_dir)
# Ademas lo guardamos en el content
#index.storage_context.persist(persist_dir="indice_jugadores")

Mounted at /content/drive


## Cargar carpeta con indices

In [ ]:
from llama_index.core import load_index_from_storage
from llama_index.core.storage import StorageContext

# Cargar el storage context desde el content
storage_context = StorageContext.from_defaults(persist_dir="indice_jugadores")
# Cargar el índice desde el contexto
index = load_index_from_storage(storage_context)
# Crear el query engine normalmente
query_engine = index.as_query_engine(similarity_top_k=3)

Loading llama_index.core.storage.kvstore.simple_kvstore from indice_jugadores/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from indice_jugadores/index_store.json.


## Consulta usando RAG

In [ ]:
print("🧠 Ejemplos de preguntas que podes hacer:")
print("- ¿Cual es la valoracion de Lionel Messi?")
print("- ¿Cual es el potencial de Kylian Mbappe?")
print("- ¿Que posicion juega Emiliano Martinez?")
print("- ¿Cual es el ritmo de Kylian Mbappe?")
print("- ¿Cuanta defensa tiene Nicolás Hernán Gonzalo Otamendi?")
print("- ¿Cual es la altura de Cristiano Ronaldo?")
print("- ¿Que pie prefiere Angel Di Maria?")
while True:
    consulta = input("\n🔎 Ingresá tu consulta (o escribí 'salir' para terminar):\n> ")

    if consulta.lower() == "salir":
        print("👋 Hasta luego.")
        break

    consulta_modificada = consulta.strip() + ". Responde en español."

    respuesta_rag = query_engine.query(consulta_modificada)
    response = llm.invoke(consulta_modificada)
    print("💬 Sin RAG:\n", response.content)
    print("\n✅ Respuesta con RAG:")
    print(str(respuesta_rag))

🧠 Ejemplos de preguntas que podes hacer:
- ¿Cual es la valoracion de Lionel Messi?
- ¿Cual es el potencial de Kylian Mbappe?
- ¿Que posicion juega Emiliano Martinez?
- ¿Cual es el ritmo de Kylian Mbappe?
- ¿Cuanta defensa tiene Nicolás Hernán Gonzalo Otamendi?
- ¿Cual es la altura de Cristiano Ronaldo?
- ¿Que pie prefiere Angel Di Maria?

🔎 Ingresá tu consulta (o escribí 'salir' para terminar):
> Que posicion juega Emiliano Martinez?
💬 Sin RAG:
 Emiliano Martínez juega como **portero** (o guardameta).

✅ Respuesta con RAG:
Emiliano Martínez juega de GK (arquero).

🔎 Ingresá tu consulta (o escribí 'salir' para terminar):
> salir
👋 Hasta luego.
